In [ ]:
import ISS_processing.preprocessing as pp
import os
import pandas as pd

This notebook guides you through the preprocessing of CZI files produced by Zeiss microscopes. The main difference in this case compared to Leica, is that we don't have a main wrapper function and the different functions need to be run one by one. The typical order by which you would need to run these function is:

`process_czi`

`zen_OME_tiff`

`ashlar_wrapper`

`tile_stitched_images`

## `process_czi`

In a typical Zeiss experiment you will have 1 single CZI file per region per cycle. These files need to be processed individually, but of course feel free to wrap this function and the others in a loop for more automated processing. This function extracts the images, organises them, and create maximum projections that are exported with a naming convention fitting the downstream processing steps.

It also parses the metadata and converts them for downstream processing.


`process_czi` takes as inputs the following arguments:

`input_file`: the path to the CZI file that you want to preprocess, down to the czi file (included)

`outpath`: the folder where you want to save the maximum-projected images. Ideally this would be a `/mainoutputfolder/region/preprocessing/mipped/` folder structure, for consistency with our way of organising the data.

`cycle`: here you have to manually specify to which ISS cycle the images refer to. This is a `int` number, where 1 refers to cycle 1 and so on. If `cycle=0` the function will not work.

`tile_size_x` and `tile_size_y`: these refer to the size in pixel of your camera field of view. Most cameras are 2048x2048, so that's the default if you don't specify them, but adjust them if your camera has a different field of view size.

In [ ]:
pp.process_czi(input_file, outpath, cycle=0)

It is required to run the `process_czi` for all the cycles of a specific sample (specifying each time the appropriate `cycle` argument, before moving to the next step.

## zen_OME_tiff

This is the function that **takes the projected images across channels and wraps them into a single OMEtiff per imaging cycle**. This steps organises the files corresponding to each imaging cycles in a specific way within a single file and requires the parsing of a Metadata file to arrange correctly the images in xy space. As explained before, the input for this function is the specific sub-folder within `output_location` one wishes to process. **This function does not allow to process multiple regions in one go, and needs to be run on individual regions manually**.
From the previous function step, the mipped images will typically have this name format:
`Base_1_c1m01_ORG.tif`

Where: `Base` refers to the cycle number, `c` refers to the channel number, `m` to the tile number. The `_` separator is used to parse the information about each image from the filename.

`Base_1_c1m01_ORG.tif` will be split into `Base_1` `c1` `m01`. Position 1 indicates the cycle, position 2 indicates the channel in this case.

The `zen_OME_tiff` accepts the following arguments:

`exported_directory`: this is the directory containing the maximum-projected images, however they were generated. They will serve as the input files

`output_directory`: this is the output directory that will contain the OMEtiff files created by the function

`channel_split`: this specifies where the channel number is indicated in the filename (default=2)

`cycle_split`: this specifies where the cycle number is indicated in the filename (default=1)

`num_channels`: this specifies how many channels (DAPI included) the images have (default=5)




In [ ]:
import ISS_processing.preprocessing as pp
sample='/mainoutputfolder/region/'

exported_directory = sample+'/preprocessing/mipped/', 
output_directory = sample+'/preprocessing/OME_tiffs/'

pp.zen_OME_tiff(exported_directory, output_directory, channel_split=2, cycle_split=1, num_channels=5)

### `ashlar_wrapper`

This function runs `ashlar`, a package for image stitching and cycle alignment. The function uses the OME_tiffs files as an input, takes as input a channel number (normally the DAPI, see above) and on that channels performs all the alignment and stitching operations.

The DAPI channel number is relative **to the order of acquisition in the microscope**, in our Zeiss microscope DAPI is the first imaged channel, but often that's not the case. Make sure to adjust this variable accordingly

The function outputs 1 stitched file per cycle and channel into the `/preprocessing/stitched/` subfolder

In [ ]:
import ISS_processing.preprocessing as pp
sample='/mainoutputfolder/region/'

OME_tiffs = os.listdir(sample+'/preprocessing/OME_tiffs/')
OME_tiffs.sort()
OME_tiffs = [sample+'/preprocessing/OME_tiffs/' + sub for sub in OME_tiffs]

pp.ashlar_wrapper(
    files = OME_tiffs,
    output = sample+'/preprocessing/stitched/',
    align_channel=0
)

### `tile_stitched_images`
In this function the stitched images are re-tiled according to a user-specified size.
The reason for this is that stitched images are too big to be decoded directly and we prefer to decode them in tiles. This has several advantages, most notably that the pipeline would work also on laptops or non-powerful computers. The idea tile size is 4000-6000, but larger or smaller are also fine depending on the computer.

In [ ]:
import ISS_processing.preprocessing as pp
sample='/mainoutputfolder/region/'

# retile stitched images
pp.tile_stitched_images(
    image_path = sample+'/preprocessing/stitched/',
    outpath = path+'/preprocessing/ReslicedTiles/',
    tile_dim=2000
)